In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import numpy as np
folder = "/data/AIpep/"
from Levenshtein import distance as lev_dist

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Filtering

In [2]:
df_anticancer = pd.read_pickle(folder+"pickles/gen_anticancer_with_NN_prop_helicity.pkl")
anticancer = df_anticancer.query("isPredActive == True")

In [3]:
len(anticancer)

11458

In [4]:
anticancer_filtered = anticancer.query("fraction_PredHelical > 0.8 and HydroMoment > 0.3 and (dist_Training > 5 and dist_Training <= 7) and dist_Test > 4 and length <= 15 and D_AA == False").reset_index(drop=True)

In [5]:
len(anticancer_filtered)

202

In [6]:
cols = ['ID', 'Sequence', 'prediction', 'length', 
       'dist_Training', 'NN_Training', 'dist_Test', 'NN_Test', 'map_dist_Training',
       'map_NN_Training', 'map_dist_Test', 'map_NN_Test', 'fraction_PredHelical', 'HydroMoment', 'SMILES', 'MAP4']

In [8]:
anticancer_filtered[cols].to_csv(folder+"data/anticancer_filtered.csv", index=False)
anticancer_filtered[cols].to_csv("data/anticancer_filtered.csv", index=False)

# Clustering

In [7]:
from rdkit.ML.Cluster.Butina import ClusterData
clusters = ClusterData(anticancer_filtered.Sequence.to_list(), len(anticancer_filtered), 10, distFunc=lev_dist)

In [8]:
len(clusters)

14

In [12]:
idx = []
for cluster in clusters:
    idx.append(cluster[0])

In [13]:
idx

[22, 105, 113, 167, 164, 88, 35, 141, 49, 195, 84, 131, 39, 143]

In [14]:
anticancer_filtered.iloc[idx][cols].to_csv(folder+"data/anticancer_filtered_clustered.csv", index=False)
anticancer_filtered.iloc[idx][cols].to_csv("data/anticancer_filtered_clustered.csv", index=False)